The goal of this notebook is to replicate the work of Bauer1999 (Bauer, D., & Mulser, P. (1999). Exact field ionization rates in the barrier-suppression regime from numerical time-dependent Schrödinger-equation calculations. Physical Review A, 59(1), 569–577. https://doi.org/10.1103/PhysRevA.59.569) and investigate correspondence between their results and our IDE model.

In [ ]:
import logging

import simulacra as si
import simulacra.units as u

import ionization as ion

LOGMAN = si.utils.LogManager('simulacra', 'ionization', stdout_level = logging.INFO)

PLOT_KWARGS = dict(
    show = True,
    save = False,
)

%matplotlib inline

This is the laser pulse used by Bauer1999. It is not centered at $t=0$, but instead at half the number of cycles times the cycle time (i.e., the number of periods and the period). Note that $\varphi = 0$ is a sine-like pulse because of this time shift. 

In [ ]:
class BauerGaussianPulse(ion.potentials.UniformLinearlyPolarizedElectricPotential):
    """Gaussian pulse as defined in Bauer1999. Phase = 0 is a sine-like pulse."""

    def __init__(
            self,
            amplitude = 0.3 * u.atomic_electric_field,
            omega = .2 * u.atomic_angular_frequency,
            number_of_cycles = 6,
            phase = 0,
            **kwargs):
        super().__init__(**kwargs)

        self.amplitude = amplitude
        self.omega = omega
        self.number_of_cycles = number_of_cycles
        self.phase = phase

        self.pulse_center = number_of_cycles * pi / self.omega
        self.sigma2 = (self.pulse_center ** 2 / (4 * np.log(20)))

    @property
    def cycle_time(self):
        return 2 * self.pulse_center / self.number_of_cycles

    def get_electric_field_envelope(self, t):
        return np.exp(-((t - self.pulse_center) ** 2) / (4 * self.sigma2))

    def get_electric_field_amplitude(self, t):
        amp = self.get_electric_field_envelope(t) * np.sin((self.omega * t) + self.phase)

        return amp * self.amplitude * super().get_electric_field_amplitude(t)
